<a href="https://colab.research.google.com/github/turnede0/PCB/blob/Tensorflow_Model/(Don't_edit_copy_first)_CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
#import library and important functions in this line
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
print(tf.version.VERSION)

2.9.2


In [43]:
!unzip -u "data.zip"
!unzip -u "test.zip"
!unzip -u "calib_data.zip"

Archive:  data.zip
Archive:  test.zip
Archive:  calib_data.zip


In [44]:
#to import dataset and split them to train val
img_height = 224
img_width = 224
batch_size = 32

#split load dataset into training images and preprocess it
train_images = tf.keras.preprocessing.image_dataset_from_directory(
    'data',
    labels='inferred',
    label_mode="int",
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_height,img_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="training"
    )
train_image = tf.keras.preprocessing.image_dataset_from_directory(
    'calib_data',
    labels='inferred',
    label_mode="int",
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_height,img_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="training"
    )
test_images = tf.keras.preprocessing.image_dataset_from_directory(
	'test',
	labels='inferred',
	label_mode="int",
	color_mode='rgb',
	batch_size=batch_size,
	image_size=(img_height, img_width),
	shuffle=True,
	seed=123,
	validation_split=0.99,
	subset="validation"
	)	

valid_images = tf.keras.preprocessing.image_dataset_from_directory(
    'data',
    labels='inferred',
    label_mode="int",
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_height,img_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="validation"
    ) 

#split load dataset into training images and preprocess it
# done load images

Found 1170 files belonging to 2 classes.
Using 936 files for training.
Found 210 files belonging to 2 classes.
Using 168 files for training.
Found 130 files belonging to 2 classes.
Using 128 files for validation.
Found 1170 files belonging to 2 classes.
Using 234 files for validation.


In [45]:
resize_and_rescale = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255)
])

In [46]:
AUTOTUNE = tf.data.AUTOTUNE
train_images = train_images.map(lambda x, y: (resize_and_rescale(x), y), 
              num_parallel_calls=AUTOTUNE)
test_images = test_images.map(lambda x, y: (resize_and_rescale(x), y), 
             num_parallel_calls=AUTOTUNE)
valid_images = valid_images.map(lambda x, y: (resize_and_rescale(x), y), 
              num_parallel_calls=AUTOTUNE)

In [47]:
model = tf.keras.models.Sequential([
    # since Conv2D is the first layer of the neural network, we should also specify the size of the input
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    
    # and repeat the process

    tf.keras.layers.Conv2D(32, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    

    # flatten the result to feed it to the dense layer 
    tf.keras.layers.Flatten(), 
    # and define 512 neurons for processing the output coming by the previous layers
    tf.keras.layers.Dense(64, activation='relu'), 
    
    # a single output neuron. The result will be 0 if the image is a cat, 1 if it is a dog
    tf.keras.layers.Dense(2, activation='softmax')  
])

In [48]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 222, 222, 16)      448       
                                                                 
 conv2d_16 (Conv2D)          (None, 220, 220, 16)      2320      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 110, 110, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 108, 108, 32)      4640      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 54, 54, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_18 (Conv2D)          (None, 52, 52, 64)       

In [49]:
model.compile(optimizer="adam",
              loss='SparseCategoricalCrossentropy',
              #loss='binary_crossentropy',
              metrics = ['accuracy'])

In [50]:
#train the model
model.fit(train_images,
          epochs=10,
          validation_data=valid_images,
          )

Epoch 1/10
30/30 [==============================] - 3s 71ms/step - loss: 0.7186 - accuracy: 0.5246 - val_loss: 0.6783 - val_accuracy: 0.5897
Epoch 2/10
30/30 [==============================] - 2s 65ms/step - loss: 0.5309 - accuracy: 0.7233 - val_loss: 0.4650 - val_accuracy: 0.7521
Epoch 3/10
30/30 [==============================] - 2s 65ms/step - loss: 0.4093 - accuracy: 0.8066 - val_loss: 0.5189 - val_accuracy: 0.7692
Epoch 4/10
30/30 [==============================] - 2s 64ms/step - loss: 0.3343 - accuracy: 0.8515 - val_loss: 0.3437 - val_accuracy: 0.8376
Epoch 5/10
30/30 [==============================] - 2s 65ms/step - loss: 0.2463 - accuracy: 0.8996 - val_loss: 0.3294 - val_accuracy: 0.8675
Epoch 6/10
30/30 [==============================] - 2s 65ms/step - loss: 0.2273 - accuracy: 0.9167 - val_loss: 0.3217 - val_accuracy: 0.8590
Epoch 7/10
30/30 [==============================] - 2s 65ms/step - loss: 0.1871 - accuracy: 0.9209 - val_loss: 0.2144 - val_accuracy: 0.9060
Epoch 8/10
30

In [51]:
model.fit(train_images,
          epochs=5,
          validation_data=valid_images,
          )

Epoch 1/5
30/30 [==============================] - 2s 65ms/step - loss: 0.1221 - accuracy: 0.9541 - val_loss: 0.2045 - val_accuracy: 0.9188
Epoch 2/5
30/30 [==============================] - 2s 65ms/step - loss: 0.0943 - accuracy: 0.9637 - val_loss: 0.2704 - val_accuracy: 0.9017
Epoch 3/5
30/30 [==============================] - 2s 65ms/step - loss: 0.1647 - accuracy: 0.9338 - val_loss: 0.2026 - val_accuracy: 0.9316
Epoch 4/5
30/30 [==============================] - 2s 65ms/step - loss: 0.0561 - accuracy: 0.9808 - val_loss: 0.3809 - val_accuracy: 0.8974
Epoch 5/5
30/30 [==============================] - 2s 65ms/step - loss: 0.0630 - accuracy: 0.9733 - val_loss: 0.2225 - val_accuracy: 0.9103


In [52]:
test_loss, test_acc = model.evaluate(test_images, verbose=2)

4/4 - 0s - loss: 1.0495 - accuracy: 0.7344 - 183ms/epoch - 46ms/step


In [53]:
#save the model
model.save("model.h5")

In [54]:
preds = model.predict(train_image)
print(preds)

6/6 [==============================] - 0s 21ms/step
[[0.0000000e+00 1.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [3.0332753e-03 9.9696678e-01]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.

In [55]:
classes = np.argmax(preds, axis = 1)
print(classes)

[1 0 1 0 1 0 0 1 1 0 0 0 1 0 1 1 0 1 1 1 1 0 0 1 0 0 1 0 1 0 0 1 1 1 1 1 0
 0 0 1 1 1 1 1 1 1 1 0 1 0 0 1 1 0 0 1 0 0 0 0 1 0 1 0 0 1 1 1 1 0 0 1 1 0
 1 1 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 1 1 1 0 1 0 1 1 0 0 1 0 1 1 0 1 1 1 0 0
 0 1 0 1 1 1 1 1 1 1 1 0 0 1 0 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 0 1
 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 0]
